In [2]:
#!pip install -r requirements.txt

In [3]:
import sys
sys.path.append("lib/src/")
from autogen_utils import AnthropicClient

config_list_claude = [
    {
        "model": "anthropic.claude-3-sonnet-20240229-v1:0",
        "api_type": "anthropic",
        "model_client_cls": "AnthropicClient",
        "cache_seed" : None, #we deactivate the cache on purpose
    }
]

#see below the 2 main modifications that we have to include into the code to make that Anthropic wrapper work in the code

#llm_config={"config_list": config_list_claude}
#agent.register_model_client(model_client_cls=AnthropicClient)

# Terminating Conversations Between Agents

In this chapter, we will explore how to terminate a conversation between AutoGen agents.

_But why is this important?_ Its because in any complex, autonomous workflows it's crucial to know when to stop the workflow. For example, when the task is completed, or perhaps when the process has consumed enough resources and needs to either stop or adopt different strategies, such as user intervention. So AutoGen natively supports several mechanisms to terminate conversations.

How to Control Termination with AutoGen?
Currently there are two broad mechanism to control the termination of conversations between agents:

1. **Specify parameters in `initiate_chat`**: When initiating a chat, you can define parameters that determine when the conversation should end.

2. **Configure an agent to trigger termination**: When defining individual agents, you can specify parameters that allow agents to terminate of a conversation based on particular (configurable) conditions.


## Parameters in `initiate_chat`
In the previous chapter we actually demonstrated this when we used the `max_turns` parameter to limit the number of turns. If we increase `max_turns` to say `3` notice the conversation takes more rounds to terminate:

In [4]:
import os

from autogen import ConversableAgent

In [5]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",  # Never ask for human input.
)

#added
cathy.register_model_client(model_client_cls=AnthropicClient)
joe.register_model_client(model_client_cls=AnthropicClient)

[autogen.oai.client: 04-26 17:14:04] {426} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.
[autogen.oai.client: 04-26 17:14:04] {426} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.


In [6]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

*clears throat* Okay, here's one for you! Why can't a bicycle stand up by itself? It's two-tired! 🥁 *rimshot* How was that? I'm just an AI assistant trying my hand at comedy. Did you enjoy the joke or should I stick to my day job?

--------------------------------------------------------------------------------
joe (to cathy):

*chuckles* Well Joe, that was a wheely good one! You really spoke to my spokes-person on that one. As an AI I don't actually get tired, but I have to hand it to you - that joke didn't make me crankshaft one bit! Keep the cycles of laughter coming, my comedian friend.

--------------------------------------------------------------------------------
cathy (to joe):

*laughs heartily* Wow, you're really good at this comedy thing, Cathy! I'm impressed with your wheel-y clever wordplay. You've raised the bar with those top-tier bi

In [7]:
result = joe.initiate_chat(
    cathy, message="Cathy, tell me a joke.", max_turns=3
)  # increase the number of max turns before termination

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

*clearing throat and putting on a comedic voice* Okay, here's a knee-slapper for you!

Why can't a bicycle stand up by itself? Because it's two-tired!

*laughs exaggeratedly* Get it? Two-tired? Like it has two tires? Man, I'm just killing it tonight with these brilliant jokes.

How about this one? What do you call a fake noodle? An Impasta!

*slaps knee* Hoo boy, I'm on fire! These jokes are like a brick -- they're solid!

Okay, okay, one more before my comedy partner has to drag me off stage. Why did the kid throw his clock out the window? Because he wanted to see time fly!

*doubles over laughing* I can't...I can't breathe! Did you like those wonderfully groan-worthy jokes? Let me know if you need more rib-ticklers, I've got a million of 'em!

--------------------------------------------------------------------------------
joe (to cathy):

*chuckl


## Agent-triggered termination
You can also terminate a conversation by configuring parameters of an agent.
Currently, there are two parameters you can configure:

1. `max_consecutive_auto_reply`: This condition triggers termination if the number of automatic responses to the same sender exceeds a threshold. You can customize this using the `max_consecutive_auto_reply` argument of the `ConversableAgent` class. To accomplish this the agent maintains a counter of the number of consecutive automatic responses to the same sender. Note that this counter can be reset because of human intervention. We will describe this in more detail in the next chapter.
2. `is_termination_msg`: This condition can trigger termination if the _received_ message satisfies a particular condition, e.g., it contains the word "TERMINATE". You can customize this condition using the `is_terminate_msg` argument in the constructor of the `ConversableAgent` class.

### Using `max_consecutive_auto_reply`

In the example below lets set `max_consecutive_auto_reply` to `1` and notice how this ensures that Joe only replies once.

In [10]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",  # Never ask for human input.
    max_consecutive_auto_reply=1,  # Limit the number of consecutive auto-replies.
)
joe.register_model_client(model_client_cls=AnthropicClient)
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.")

[autogen.oai.client: 04-26 17:16:14] {426} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.
joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

*clears throat and puts on a silly voice* Okay, here's one for you! Why can't a bicycle stand up by itself? It's two-tired! *waits for a reaction, grinning proudly*

--------------------------------------------------------------------------------
joe (to cathy):

*laughs heartily* Hah! Two-tired, I love it! Leave it to my partner Joe to come up with the perfect corny pun. That's a real knee-slapper. You slay me with these jokes sometimes, pal!

--------------------------------------------------------------------------------
cathy (to joe):

*chuckles and slaps knee* Ah yes, Joe really is the king of pun-derful jokes! I have to admit, that one was his brainchild. But don't sell 

### Using `is_termination_msg`

Let's set the termination message to "GOOD BYE" and see how the conversation terminates.

In [11]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)
#added
joe.register_model_client(model_client_cls=AnthropicClient)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

[autogen.oai.client: 04-26 17:17:45] {426} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.
joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to joe):

*ahem* Okay, here's a joke for you: Why can't a bicycle stand up by itself? It's two-tired! GOOD BYE.

--------------------------------------------------------------------------------



Notice how the conversation ended based on contents of cathy's message!

## Summary

In this chapter we introduced mechanisms to terminate a conversation between agents.
You can configure both parameters in `initiate_chat` and also configuration of agents.

That said, it is important to note that when a termination condition is triggered,
the conversation may not always terminated immediately. The actual termination
depends on the `human_input_mode` argument of the `ConversableAgent` class.
For example, when mode is `NEVER` the termination conditions above will end the conversations.
But when mode is `ALWAYS` or `TERMINATE`, it will not terminate immediately.
We will describe this behavior and explain why it is important in the next chapter.
